In [73]:
import pandas as pd
import numpy as np

df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')

C:\Users\alova\AppData\Local\Temp\ipykernel_996\2594960417.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')


In [74]:
df_people.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID',
       'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
       'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE',
       'CELL_PHONE_USE'],
      dtype='object')

In [75]:
#Seleccionamos las columnas a usar
df_people_driver_info = df_people[['PERSON_ID', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT VALUE', 'CELL_PHONE_USE', 'DRIVERS_LICENSE_CLASS']]
df_people_driver_info.shape

(2190983, 7)

In [76]:
#cambiamos el nombre de la columna BAC_RESULT VALUE para estandarizar
df_people_driver_info.rename(columns = {'BAC_RESULT VALUE' : 'BAC_RESULT_VALUE'})

,PERSON_ID,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT_VALUE,CELL_PHONE_USE,DRIVERS_LICENSE_CLASS
0,O2179723,UNKNOWN,NOT OBSCURED,NORMAL,NaN,NaN,D
1,P480828,NaN,NaN,NaN,NaN,NaN,NaN
2,P480829,NaN,NaN,NaN,NaN,NaN,NaN
3,O2179719,OTHER,UNKNOWN,UNKNOWN,NaN,NaN,NaN
4,O2179720,NONE,NOT OBSCURED,NORMAL,NaN,NaN,D
...,...,...,...,...,...,...,...
2190978,O1399482,UNKNOWN,UNKNOWN,NORMAL,NaN,NaN,NaN
2190979,O1399483,NONE,NOT OBSCURED,NORMAL,NaN,NaN,D
2190980,O595730,FAILED TO YIELD,UNKNOWN,UNKNOWN,NaN,NaN,NaN
2190981,O595731,NONE,NOT OBSCURED,NORMAL,NaN,NaN,D


In [77]:
for c in df_people_driver_info:
    print("------- Atributo:", c)
    print("Valores nulos: ", df_people_driver_info[c].isna().sum())
    print("Valores distintos: ", df_people_driver_info[c].unique())


------- Atributo: PERSON_ID
Valores nulos:  0
Valores distintos:  ['O2179723' 'P480828' 'P480829' ... 'O595730' 'O595731' 'P134373']
------- Atributo: DRIVER_ACTION
Valores nulos:  446163
Valores distintos:  ['UNKNOWN' nan 'OTHER' 'NONE' 'OVERCORRECTED' 'FOLLOWED TOO CLOSELY'
 'FAILED TO YIELD' 'IMPROPER BACKING' 'IMPROPER TURN' 'IMPROPER PASSING'
 'DISREGARDED CONTROL DEVICES' 'EVADING POLICE VEHICLE'
 'IMPROPER LANE CHANGE' 'TOO FAST FOR CONDITIONS'
 'CELL PHONE USE OTHER THAN TEXTING' 'EMERGENCY VEHICLE ON CALL'
 'WRONG WAY/SIDE' 'TEXTING' 'IMPROPER PARKING' 'STOPPED SCHOOL BUS'
 'LICENSE RESTRICTIONS']
------- Atributo: DRIVER_VISION
Valores nulos:  446876
Valores distintos:  ['NOT OBSCURED' nan 'UNKNOWN' 'PARKED VEHICLES' 'MOVING VEHICLES'
 'BLINDED - SUNLIGHT' 'OTHER' 'SIGNBOARD' 'WINDSHIELD (WATER/ICE)'
 'BLINDED - HEADLIGHTS' 'TREES, PLANTS' 'BUILDINGS' 'EMBANKMENT'
 'HILLCREST' 'BLOWING MATERIALS']
------- Atributo: PHYSICAL_CONDITION
Valores nulos:  444978
Valores distintos: 

In [ ]:
#no hay duplicados
df_people_driver_info['PERSON_ID'].duplicated().sum()


np.int64(0)

In [78]:
df_people_driver_info.columns = (
    df_people_driver_info.columns
      .astype(str)
      .str.replace("\ufeff", "", regex=False)       
      .str.replace(r"^[^A-Za-z0-9]+", "", regex=True) 
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)         
)

In [ ]:
#convertir los datos basura a None, para que en postgres se consideren NULL
df_people_driver_info = df_people_driver_info.replace(['', "", ' ', 'NA', 'N/A', 'null', 'NULL', "UNKNOWN"], None)
df_people_driver_info = df_people_driver_info.where(df_people_driver_info.notna(), None) 

In [80]:
print(df_people_driver_info.columns)

Index(['PERSON_ID', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION',
       'BAC_RESULT_VALUE', 'CELL_PHONE_USE', 'DRIVERS_LICENSE_CLASS'],
      dtype='object')


In [81]:
import re

# limpiamos los valores inválidos del atributo DRIVERS_LICENCE_CLASS  
df_people_driver_info['DRIVERS_LICENSE_CLASS'] = (
    df_people_driver_info['DRIVERS_LICENSE_CLASS']
    .astype("string")
    .str.upper()
    .str.strip()
    .apply(lambda x: x if pd.notna(x) and re.fullmatch(r'[A-Z0-9]+', x) else None)
)

In [82]:
#verificamos
df_people_driver_info['DRIVERS_LICENSE_CLASS'].unique()

array(['D', None, 'A', 'DA', 'EO', 'DM', 'C', 'AM', 'B', 'E', 'M', 'BM',
       'O', 'DV', 'DL', '0', 'UN', 'CD', 'N', 'CM', 'TM', 'U', 'IL', 'UK',
       'R', 'NO', 'CA', 'DD', 'F', '1', 'AB', 'AR', 'AL', 'K', 'LP', 'H',
       'G', '9', 'WA', 'D7', 'PS', 'L', 'C2', 'J', 'S', 'P', '99', 'MD',
       'C1', 'HN', 'MC', 'I', 'DC', '10', 'A3', 'D9', 'CH', 'DE', 'XD',
       'M2', 'CL', '1F', 'TV', 'MP', '5', 'X', 'BF', 'BC', 'IP', 'DN',
       'LN', 'BW', 'V', '3', 'EC', 'DR', 'KM', 'TL', 'NT', 'ND', 'BE',
       'AC', '00', 'G1', 'TD', 'DQ', 'OA', 'P2', 'D5', 'NP', 'Y', 'RG',
       'CP', 'T', 'DS', 'RE', 'UB', 'NC', '2', 'ID', 'SB', 'CB', 'NV',
       'AA', 'CO', 'NM', 'W', 'BP', 'TP', 'Q', 'P1', '4', 'PR', 'AP',
       'ST', 'DF', 'VL', '6', 'BL', 'GM', 'LT', '25', 'DJ', 'A0', 'DB',
       'PA', 'Z', 'XX', 'RF', 'B1', 'PV', 'EM', 'M1', '4D', 'DH', '1B',
       'DU', '01', 'E0', 'FD', 'JR', 'DZ', 'DW', '05', 'BG', '2D', 'AD',
       'A1', 'MO', 'SC', 'LM', 'CQ', '56', '31', 'EX', 'SD', 

In [83]:
#reemplazamos los valores de unknown en DRIVER ACTION 
df_people_driver_info['DRIVER_ACTION'] = df_people_driver_info['DRIVER_ACTION'].replace("UNKNOWN", None)
df_people_driver_info['DRIVER_ACTION'].unique()

array([None, 'OTHER', 'NONE', 'OVERCORRECTED', 'FOLLOWED TOO CLOSELY',
       'FAILED TO YIELD', 'IMPROPER BACKING', 'IMPROPER TURN',
       'IMPROPER PASSING', 'DISREGARDED CONTROL DEVICES',
       'EVADING POLICE VEHICLE', 'IMPROPER LANE CHANGE',
       'TOO FAST FOR CONDITIONS', 'CELL PHONE USE OTHER THAN TEXTING',
       'EMERGENCY VEHICLE ON CALL', 'WRONG WAY/SIDE', 'TEXTING',
       'IMPROPER PARKING', 'STOPPED SCHOOL BUS', 'LICENSE RESTRICTIONS'],
      dtype=object)

In [84]:
#reemplazamos los valores de unknown en driver vision
df_people_driver_info['DRIVER_VISION'] = df_people_driver_info['DRIVER_VISION'].replace("UNKNOWN", None)
df_people_driver_info['DRIVER_VISION'].unique()

array(['NOT OBSCURED', None, 'PARKED VEHICLES', 'MOVING VEHICLES',
       'BLINDED - SUNLIGHT', 'OTHER', 'SIGNBOARD',
       'WINDSHIELD (WATER/ICE)', 'BLINDED - HEADLIGHTS', 'TREES, PLANTS',
       'BUILDINGS', 'EMBANKMENT', 'HILLCREST', 'BLOWING MATERIALS'],
      dtype=object)

In [85]:
#reemplazamos los valores de unknown 
df_people_driver_info['PHYSICAL_CONDITION'] = df_people_driver_info['PHYSICAL_CONDITION'].replace("UNKNOWN", None)
df_people_driver_info['PHYSICAL_CONDITION'].unique()

array(['NORMAL', None, 'IMPAIRED - ALCOHOL', 'EMOTIONAL',
       'IMPAIRED - DRUGS', 'OTHER', 'HAD BEEN DRINKING',
       'FATIGUED/ASLEEP', 'REMOVED BY EMS',
       'IMPAIRED - ALCOHOL AND DRUGS', 'ILLNESS/FAINTED', 'MEDICATED'],
      dtype=object)

In [105]:
df_people_driver_info['CELL_PHONE_USE'].unique()
df_people_driver_info['CELL_PHONE_USE'] = df_people_driver_info['CELL_PHONE_USE'].replace('Y', True)
df_people_driver_info['CELL_PHONE_USE'] = df_people_driver_info['CELL_PHONE_USE'].replace('N', False)

#comprobamos
print(df_people_driver_info['CELL_PHONE_USE'].unique())
print((df_people_driver_info['CELL_PHONE_USE'] == True).sum())
print((df_people_driver_info['CELL_PHONE_USE'] == False).sum())
print((df_people_driver_info['CELL_PHONE_USE'].isna()).sum())


print("Porcentaje de valores verdaderos:", (df_people_driver_info['CELL_PHONE_USE'] == False).sum()/len(df_people_driver_info))
print("Porcentaje de valores falsos:", (df_people_driver_info['CELL_PHONE_USE'] == True).sum()/len(df_people_driver_info))


<BooleanArray>
[<NA>, True, False]
Length: 3, dtype: boolean
754
408
2189821
Porcentaje de valores verdaderos: 0.00018621778443739636
Porcentaje de valores falsos: 0.0003441377682985217


In [ ]:
df_people_driver_info['BAC_RESULT_VALUE'].unique()
df_people_driver_info['BAC_RESULT_VALUE'].isna().sum() / len(df_people_driver_info)

np.float64(0.9989123603423669)

In [97]:
#Cambiamos los tipos de datos 
df_people_driver_info = df_people_driver_info.assign(
    PERSON_ID = df_people_driver_info['PERSON_ID'].astype('string'),
    DRIVER_ACTION = df_people_driver_info['DRIVER_ACTION'].astype('string'),
    DRIVER_VISION = df_people_driver_info['DRIVER_VISION'].astype('string'),
    PHYSICAL_CONDITION = df_people_driver_info['PHYSICAL_CONDITION'].astype('string'),
    BAC_RESULT_VALUE = df_people_driver_info['BAC_RESULT_VALUE'].astype('string'),
    CELL_PHONE_USE = df_people_driver_info['CELL_PHONE_USE'].astype('boolean'),
    DRIVERS_LICENSE_CLASS = df_people_driver_info['DRIVERS_LICENSE_CLASS'].astype('string')
)

In [ ]:
for c in df_people_driver_info:
    print(c, "Nulls: ", df_people_driver_info[c].isna().sum() / len(df_people_driver_info))

PERSON_ID Nulls:  0.0
DRIVER_ACTION Nulls:  0.4073021105138652
DRIVER_VISION Nulls:  0.5916691275103458
PHYSICAL_CONDITION Nulls:  0.4722181778681076
BAC_RESULT_VALUE Nulls:  0.9989123603423669
CELL_PHONE_USE Nulls:  0.999469644447264
DRIVERS_LICENSE_CLASS Nulls:  0.5154846021169494


In [ ]:
df_people_driver_info.to_csv("people_driver_info.csv")


In [ ]:
df_people_driver_info

,PERSON_ID,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT_VALUE,CELL_PHONE_USE,DRIVERS_LICENSE_CLASS
0,O2179723,<NA>,NOT OBSCURED,NORMAL,<NA>,<NA>,D
1,P480828,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,P480829,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,O2179719,OTHER,<NA>,<NA>,<NA>,<NA>,<NA>
4,O2179720,NONE,NOT OBSCURED,NORMAL,<NA>,<NA>,D
...,...,...,...,...,...,...,...
2190978,O1399482,<NA>,<NA>,NORMAL,<NA>,<NA>,<NA>
2190979,O1399483,NONE,NOT OBSCURED,NORMAL,<NA>,<NA>,D
2190980,O595730,FAILED TO YIELD,<NA>,<NA>,<NA>,<NA>,<NA>
2190981,O595731,NONE,NOT OBSCURED,NORMAL,<NA>,<NA>,D
